In [ ]:
# -*- coding: utf-8 -*-
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers

!pip install -q datasets trl peft bitsandbytes sentencepiece wandb

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb
from trl import setup_chat_format

Found existing installation: transformers 4.57.0.dev0
Uninstalling transformers-4.57.0.dev0:
  Successfully uninstalled transformers-4.57.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fe1tjjc_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fe1tjjc_
  Resolved https://github.com/huggingface/transformers to commit 946e5f95ea0894e5c88f7915547ed3ffcafa56cd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.57.0.dev0-py3-none-any.whl size=11477606 sha256=88e7113b1d1c68b55026dd7fe3301358c945d8f0a9bb4ebb0118ce6beed0d937
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xs3rb2j/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers


In [ ]:
import huggingface_hub

In [ ]:
huggingface_hub.login()

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
# Defined in the secrets tab in Google Colab
hf_token = userdata.get('HF_TOKEN')
wb_token = ''
wandb.login(key=wb_token)


In [ ]:

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, setup_chat_format, SFTConfig

In [ ]:
# Model
base_model = "HuggingFaceTB/SmolLM2-135M"


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True, trust_remote_code=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=base_model
)
model.to(device)
print(device)
# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

cuda


In [ ]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-SFT"
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    #evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    processing_class=tokenizer,
    eval_dataset=ds["test"],
)

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
!huggingface-cli login

In [ ]:
model.push_to_hub(finetune_name, use_temp_dir=False)
tokenizer.push_to_hub(finetune_name, use_temp_dir=False)